# Imports

In [1]:
import pandas as pd
import os 
current_folder = os.getcwd()
print(current_folder)
parent_folder = os.path.dirname(current_folder)
os.chdir(parent_folder)

/home/araceli/code/energy-analysis-and-predictions/notebooks


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from utils.parameters import ENERGY_COMPANIES_FOLDER, ENERGY_COMPANY_SECTOR

from load_data.load_energy_prices import load_energy_prices_data, \
                            load_energy_prices_from_all_files         

from predict_data.create_train_test_data import get_scaled_X_y_train_and_test_data
from predict_data.create_simple_train_test_data import get_X_y_scaled_and_splited                       
from predict_data.load_gru_model import load_gru_model          
from predict_data.model_methods import compile_model, train_model,evaluate_model, predict_model                                          

from visualize_data.plot_price_with_interactive_timestamps import plot_price_with_interactive_timestamps   
from visualize_data.plot_model_history import plot_history
from visualize_data.plot_predict_vs_test import plot_predict_vs_test

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# COMPANY SECTORS

In [53]:
energy_sectors = pd.read_csv(ENERGY_COMPANY_SECTOR ,delimiter = ",", encoding = "utf-8") 
energy_sectors.head()

,Name,Place,Energy_Source,Rated_capacity
0,Abfallentsorgungszentrum Asdonkshof,Kamp-Lintfort,Abfall,20.8 MW
1,Abfallheizkraftwerk Neunkirchen,Neunkirchen,Abfall,11.6 MW
2,AMK - Abfallentsorgungsgesellschaft des Maerki...,Iserlohn,Abfall,12.6 MW
3,AVA GmbH,Augsburg,Abfall,10 MW
4,AVA Velsen,Saarbruecken,Abfall,16 MW


In [55]:
energy_sectors['Energy_Source'] = energy_sectors['Energy_Source'].map(lambda x: str(x)[:-1])

In [56]:
energy_sector_types = energy_sectors['Energy_Source'].value_counts()
energy_sector_types

Erdgas                                    269
Laufwasser                                104
Wind (Onshore)                             90
Steinkohle                                 80
Photovoltaik                               63
Biomasse                                   56
na                                         55
Abfall                                     45
Braunkohle                                 28
Pumpspeicher                               28
Wind (Offshore)                            28
Sonstige konventionelle Energietraeger     25
Mineraloelprodukte                         23
Speicherwasser (ohne Pumpspeicher)         21
Batteriespeicher                           15
Kernenergie                                 8
Steinkohle - Heizoe                         1
Biomass                                     1
Waerme                                      1
Erdgas – Heizoe                             1
Name: Energy_Source, dtype: int64

In [57]:
sectors = list(energy_sector_types.index)
#sectors = [sector[:-1] for sector in sectors]
sectors.sort()
sectors

['Abfall',
 'Batteriespeicher',
 'Biomass',
 'Biomasse',
 'Braunkohle',
 'Erdgas',
 'Erdgas – Heizoe',
 'Kernenergie',
 'Laufwasser',
 'Mineraloelprodukte',
 'Photovoltaik',
 'Pumpspeicher',
 'Sonstige konventionelle Energietraeger',
 'Speicherwasser (ohne Pumpspeicher)',
 'Steinkohle',
 'Steinkohle - Heizoe',
 'Waerme',
 'Wind (Offshore)',
 'Wind (Onshore)',
 'na']

In [64]:
dict_companies = {sector:[] for sector in sectors}
dict_companies

{'Abfall': [],
 'Batteriespeicher': [],
 'Biomass': [],
 'Biomasse': [],
 'Braunkohle': [],
 'Erdgas': [],
 'Erdgas – Heizoe': [],
 'Kernenergie': [],
 'Laufwasser': [],
 'Mineraloelprodukte': [],
 'Photovoltaik': [],
 'Pumpspeicher': [],
 'Sonstige konventionelle Energietraeger': [],
 'Speicherwasser (ohne Pumpspeicher)': [],
 'Steinkohle': [],
 'Steinkohle - Heizoe': [],
 'Waerme': [],
 'Wind (Offshore)': [],
 'Wind (Onshore)': [],
 'na': []}

In [ ]:
Erdgas = []
Laufwasser ', 'Wind (Onshore) ', 'Steinkohle ',
       'Photovoltaik ', 'Biomasse ', 'Abfall ', 'Braunkohle ', 'Pumpspeicher ',
       'Wind (Offshore) ', 'Sonstige konventionelle Energietraeger ',
       'Mineraloelprodukte ', 'Speicherwasser (ohne Pumpspeicher) ',
       'Batteriespeicher ', 'Kernenergie ', 'Steinkohle - Heizoel', 'Biomasse',
       'Waerme ', 'Erdgas – Heizoel'

# Power Plant data

In [15]:
#foldername = os.path.join(ENERGY_COMPANIES_FOLDER,'power-plant-data-2022-Jan-to-Dec' )
power_plant_list = os.listdir(ENERGY_COMPANIES_FOLDER)
power_plant_list.sort()
print('len(power_plant_list)=', len(power_plant_list))


len(power_plant_list)= 226


In [16]:
power_plant_list[0]

'Abwinden-Asten_202201010000_202212312359_Tag_1.csv'

In [19]:
filename = os.path.join(ENERGY_COMPANIES_FOLDER, power_plant_list[0])
print(filename)
test_1 = pd.read_csv(filename, delimiter=',', decimal='.')
test_1.head()

/home/araceli/code/energy-analysis-and-predictions/raw_data/energy-companies/power-plant-data-2022-Jan-to-Dec/Abwinden-Asten_202201010000_202212312359_Tag_1.csv


,Datum,Anfang,Ende,Generation_DE [MW] Berechnete Auflösungen
0,01.01.2022,00:00,00:00,3.906
1,02.01.2022,00:00,00:00,3.813
2,03.01.2022,00:00,00:00,3.479
3,04.01.2022,00:00,00:00,3.488
4,05.01.2022,00:00,00:00,3.667


In [25]:
x=power_plant_list[0].split('_')[0]
x

'Abwinden-Asten'

In [35]:
energy_sectors['Name'].isna().value_counts()

False    922
True      20
Name: Name, dtype: int64

In [65]:
energy_sectors = energy_sectors.dropna()
condition = energy_sectors['Name'].str.contains(x)
# condition = energy_sectors['Name']==x
company_df = energy_sectors[condition]
company_sector = company_df['Energy_Source'].values[0]
print(company_sector )
#energy_sectors['Name'].str.contains(x).any()
#dict_companies[company_sector].append(test_1)
dict_companies['Abfall']

Laufwasser


[]

In [ ]:
test =  pd.read_csv('SMARD/power-plant-data-2022-Jan-to-Dec/Amrumbank_West_202201010000_202212312359_Tag_4.csv',delimiter=';', decimal=',') 
test

In [ ]:
test.shape

(1, 4)

In [ ]:
test2 = pd.read_csv('SMARD/power-plant-data-2022-Jan-to-Dec/Pumpspeicherkraftwerk_Vianden_202201010000_202212312359_Tag_137.csv', delimiter=';', decimal=',') 
test2.head()

In [ ]:
num_cols = test2.columns[3:len(test2)]

In [ ]:
def convert_str_to_float(text):
    if text == '-':
        return 0
    else:
        text=text.replace(',','.')
        return float(text)

In [ ]:
test2['Generation_DE Maschine 9 [MW] Berechnete Auflösungen']=test2['Generation_DE Maschine 9 [MW] Berechnete Auflösungen'].apply(from_str_to_float)

In [ ]:
test2['Generation_DE Maschine 9 [MW] Berechnete Auflösungen'].head()

0    148.0
1      0.0
2    257.0
3    210.0
4      0.0
Name: Generation_DE Maschine 9 [MW] Berechnete Auflösungen, dtype: float64

In [ ]:
for col in num_cols:
    test2[col] = test2[col].apply(convert_str_to_float)  

In [ ]:
test2.dtypes

Datum                                                     object
Anfang                                                    object
Ende                                                      object
Generation_DE Maschine 9 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 8 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 2 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 3 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 4 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 7 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 1 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 5 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 6 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 10 [MW] Berechnete Auflösungen    float64
Generation_DE Maschine 11 [MW] Berechnete Auflösungen    float64
dtype: object

In [ ]:
test2['Generation_DE [MW] Berechnete Auflösungen'] = test2.sum(numeric_only = True, axis =1)
test2.head()

In [ ]:
148.0 + 176.0+ 0.0+ 1.475+ 765.000+ 232.0+ 1.476+ 460.0	+ 391.0+ 0.0+ 292.0	

2466.951

In [ ]:
type(test2['Generation_DE Maschine 9 [MW] Berechnete Auflösungen'][177])

str

In [ ]:
test2['Generation_DE Maschine 9 [MW] Berechnete Auflösungen'][178]

'-'

In [ ]:
float(test2['Generation_DE Maschine 9 [MW] Berechnete Auflösungen'][177])

230.0

In [ ]:
# Take first company
file1 = power_plant_list[0].split('_')[0]
col = file1.split('_')[0]
folder = path + '/' + power_plant_list[0]
power_plant = pd.read_csv(folder, delimiter=';', decimal=',')
power_plant.columns = ['Date', 'Start', 'End', col]
power_plant.head()

,Date,Start,End,Abwinden-Asten
0,01.01.2022,00:00,00:00,3.906
1,02.01.2022,00:00,00:00,3.813
2,03.01.2022,00:00,00:00,3.479
3,04.01.2022,00:00,00:00,3.488
4,05.01.2022,00:00,00:00,3.667


In [ ]:
power_plant_list.pop(1)
power_plant_list[0:5]

['Abwinden-Asten_202201010000_202212312359_Tag_1.csv',
 'Ahrensfelde_202201010000_202212312359_Tag_2.csv',
 'Ahrensfelde_202201010000_202212312359_Tag_2.csv:Zone.Identifier',
 'Altenw_rth_202201010000_202212312359_Tag_3.csv',
 'Altenw_rth_202201010000_202212312359_Tag_3.csv:Zone.Identifier']

In [ ]:
for file in power_plant_list:
    col = file.split('_')[0]
    folder = path + '/' + file
    df = pd.read_csv(folder, delimiter=';', decimal=',')
    
    if df.shape[0] > 1:
        if df.shape[2] > 4:
            num_cols =  df.columns[3:len(test2)]
            for col in num_cols:
                test2[col] = test2[col].apply(convert_str_to_float)  
            df['Generation_DE [MW] Berechnete Auflösungen'] = df.sum(numeric_only = True, axis =1)
        data_name = 'Generation_DE [MW] Berechnete Auflösungen' 
        for i in df.columns:
            if 'Berechnete Auflösungen' in i:
                data_name = i
        power_plant[col] = df[data_name]
    else:
        print(f'Company {col} data is empty')


,Datum,Anfang,Ende,Generation_DE [MW] Berechnete Auflösungen
0,01.01.2022,00:00,00:00,3.906
1,02.01.2022,00:00,00:00,3.813
2,03.01.2022,00:00,00:00,3.479
3,04.01.2022,00:00,00:00,3.488
4,05.01.2022,00:00,00:00,3.667


In [ ]:
power_plant_dict= {
    'Laufwasser': ['Abwinden-Asten'],
    'Erdgas' : ['Ahrensfelde']    
}
    